<a href="https://colab.research.google.com/github/Yomunozs/Aprendizaje_Maquina/blob/main/Mario/m_v3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # Install virtualenv if you don't have it
# !pip install virtualenv

# # Create a virtual environment with Python 3.6
# !virtualenv -p python3.6 tf_env

# # Activate the virtual environment
# # On Windows
# tf_env\Scripts\activate
# # On macOS/Linux
# source tf_env/bin/activate

In [ ]:
# Install necessary packages with specific versions for compatibility
# !pip install gym==0.17.2
# !pip install gym-super-mario-bros==7.3.0
# !pip install nes-py==8.1.8
# !pip install tf-agents==0.5.0

In [ ]:
#!pip install tensorflow==1.15.2

In [ ]:
#!pip install --upgrade tensorflow

In [ ]:
!sudo apt-get -q update -q
!sudo apt-get install -y -q xvfb ffmpeg freeglut3-dev -q
print('DONE1')
!pip install -q 'imageio==2.4.0' -q
!pip install -q pyvirtualdisplay -q
print('DONE2')
!pip install -q tf-agents[reverb] -q
!pip install -q pyglet -q
print('DONE3')
!pip install -q swig -q
!pip install -q gym[atari,box2d,accept-rom-license] -q #install gym and virtual display
print ('DONE4')
!pip install -q gym-super-mario-bros -q
print('DONE')

In [ ]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf
import tf_agents
from tf_agents.environments import suite_gym, tf_py_environment
from tf_agents.environments.wrappers import ActionRepeat
from tf_agents.networks.q_network import QNetwork
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.utils import common
from tf_agents.replay_buffers import TFUniformReplayBuffer
from tf_agents.trajectories import trajectory
from tf_agents.policies import random_tf_policy
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver


from tf_agents.environments import gym_wrapper
from tf_agents.environments import tf_py_environment

import gym
from nes_py.wrappers import JoypadSpace
from gym.wrappers import TimeLimit
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT, RIGHT_ONLY

# To get smooth animations
import matplotlib.animation as animation
matplotlib.rc('animation', html='jshtml')

# Print versions of imported packages
print(f"imageio version: {imageio.__version__}")
print(f"pyvirtualdisplay version: {pyvirtualdisplay.__version__}")

# Print TensorFlow version separately
print(f"tensorflow version: {tf.__version__}")

# Print tf-agents version
try:
    print(f"tf_agents version: {tf_agents.__version__}")
except AttributeError:
    print("tf_gents version: not available")

# Print other versions
print(f"gym version: {gym.__version__}")
print(f"matplotlib version: {matplotlib.__version__}")
print(f"PIL version: {PIL.Image.__version__}")  # Use PIL.Image to get version

# Handle reverb
try:
    import pkg_resources
    reverb_version = pkg_resources.get_distribution("reverb").version
    print(f"reverb version: {reverb_version}")
except Exception:
    print("reverb version: not available")

# Handle gym-super-mario-bros
try:
    print(f"gym-super-mario-bros version: {gym_super_mario_bros.__version__}")
except AttributeError:
    print("gym-super-mario-bros version: not available")

In [ ]:
env = gym_super_mario_bros.make('SuperMarioBros-v2')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
print(SIMPLE_MOVEMENT)
print(COMPLEX_MOVEMENT)
print(RIGHT_ONLY)
state = env.reset()
env.step(env._action_space.sample())
img = env.render(mode="rgb_array")
plt.figure(figsize=(4, 6))
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
# Crear y envolver el entorno
def create_environment():
    env = gym_super_mario_bros.make('SuperMarioBros-v2')# Cargar el entorno de Super Mario Bros pixelado por simplificacion
    #env = TimeLimit(env, max_episode_steps=max_steps)
    env = JoypadSpace(env, SIMPLE_MOVEMENT)# usar grupo de movimientos simple
    env = gym.wrappers.GrayScaleObservation(env, keep_dim=True)  # Convertir las observaciones a escala de grises
    env = gym.wrappers.ResizeObservation(env, 84)  # Redimensionar las observaciones a 84x84 píxeles
    env = gym.wrappers.FrameStack(env, 4)  # Apilar las últimas 4 observaciones
    env = gym_wrapper.GymWrapper(env) # Envolver el entorno con un envoltorio personalizado

    # Retornar el entorno preparado
    return env
print('Done')

In [ ]:
# # Create and wrap the environment prueba diferente codigo
# max_steps = 100000
# def create_environment(max_steps=100000):
#     # Load the Super Mario Bros environment
#     env = gym_super_mario_bros.make('SuperMarioBros-1-1-v3')
# #     env = TimeLimit(env, max_episode_steps=max_steps)
#     env = JoypadSpace(env, COMPLEX_MOVEMENT)
#     env = gym.wrappers.GrayScaleObservation(env, keep_dim=True)
#     env = gym.wrappers.ResizeObservation(env, 84)
#     env = gym.wrappers.FrameStack(env, 4)
#     return env
# print('Done')

In [ ]:
# Instanciar entornos para entrenamiento y evaluación
train_py_env = create_environment()  # Crea el entorno para entrenamiento
eval_py_env = create_environment()   # Crea el entorno para evaluación

# train_py_env1 = create_environment1()  # Crea un segundo entorno para entrenamiento
# eval_py_env1 = create_environment1()   # Crea un segundo entorno para evaluación
print('Done')

In [ ]:
print("Límite de pasos por episodio:", eval_py_env.spec.max_episode_steps)

In [ ]:
# Convert to TensorFlow Agents environments
train_env = tf_py_environment.TFPyEnvironment(train_py_env)  # Convertir el entorno de entrenamiento a un entorno de TensorFlow Agents
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)    # Convertir el entorno de evaluación a un entorno de TensorFlow Agents


#train_env1 = tf_py_environment.TFPyEnvironment(train_py_env1)
#eval_env1 = tf_py_environment.TFPyEnvironment(eval_py_env1)
print('Done')

In [ ]:
# Definir la red Q
preprocessing_layer = tf.keras.layers.Lambda(lambda x: tf.cast(x, np.float32) / 255.)  # Se crea una capa de preprocesamiento para escalar las imágenes entre 0 y 1
conv_layer_params = [  # Se definen los parámetros para las capas convolucionales
    (32, (8, 8), 4),  # Se configura la primera capa convolucional (número de filtros, tamaño del kernel, paso)
    (64, (4, 4), 2),  # Se configura la segunda capa convolucional
    (64, (3, 3), 1),  # Se configura la tercera capa convolucional
]

fc_layer_params = [512]  # Se definen los parámetros para las capas totalmente conectadas 'neuronas'

# Crear la red Q
q_net = QNetwork(
    input_tensor_spec = train_env.observation_spec(),  # Se especifica la entrada del entorno de entrenamiento
    action_spec = train_env.action_spec(),  # Se especifican las acciones del entorno de entrenamiento
    preprocessing_layers = preprocessing_layer,  # Se asigna la capa de preprocesamiento
    conv_layer_params = conv_layer_params,  # Se asignan los parámetros de las capas convolucionales
    fc_layer_params = fc_layer_params  # Se asignan los parámetros de las capas totalmente conectadas
)

# Crear el agente DQN
optimizer = tf.compat.v1.train.RMSPropOptimizer(  # Se define el optimizador RMSProp
    learning_rate = 2.5e-4,  # Se establece la tasa de aprendizaje
    decay = 0.95,  # Se configura el decaimiento del optimizador
    momentum = 0.0,  # Se establece el momentum del optimizador
    epsilon = 0.01,  # Se configura el epsilon para el optimizador
    centered = True  # Se utiliza la versión centrada del RMSProp
)

train_step_counter = tf.Variable(0)  # Se inicializa un contador para los pasos de entrenamiento

# Inicializar el agente DQN
agent = DqnAgent(
    time_step_spec = train_env.time_step_spec(),  # Se especifica el tiempo del entorno de entrenamiento
    action_spec = train_env.action_spec(),  # Se especifican las acciones del entorno de entrenamiento
    q_network = q_net,  # Se asigna la red Q que se utilizará
    optimizer = optimizer,  # Se asigna el optimizador para el agente
    td_errors_loss_fn = common.element_wise_squared_loss,  # Se define la función de pérdida para errores temporales
    train_step_counter = train_step_counter,  # Se asigna el contador de pasos de entrenamiento
    gamma = 0.99,  # Se establece el factor de descuento
    epsilon_greedy = 0.1,  # Se configura la probabilidad de elegir una acción aleatoria
    target_update_period = 10000  # Se establece la frecuencia para actualizar el objetivo
)

print('Done')

In [ ]:
agent.initialize()  # Se inicializa el agente DQN

# Create the replay buffer
replay_buffer = TFUniformReplayBuffer(
    data_spec = agent.collect_data_spec,  # Se especifica la estructura de los datos que se recogerán
    batch_size = train_env.batch_size,  # Se establece el tamaño del lote para el buffer, acciones, recompensas y otros elementos relevantes.
    max_length = 100000  # Se define la longitud máxima del buffer de repetición, se actualiza
)
print('Done')

In [ ]:
# Función para recolectar experiencia
def collect_step(environment, policy, buffer):
    time_step = environment.current_time_step()  # Se obtiene el estado actual del entorno
    action_step = policy.action(time_step)  # Se calcula la acción a partir del estado actual
    next_time_step = environment.step(action_step.action)  # Se aplica la acción en el entorno y se obtiene el siguiente estado
    traj = trajectory.from_transition(time_step, action_step, next_time_step)  # Se crea una trayectoria a partir de la transición
    buffer.add_batch(traj)  # Se añade la trayectoria al buffer de experiencias

# Se recolectan datos iniciales con una política aleatoria
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(), train_env.action_spec())  # Se define una política aleatoria
initial_collect_steps = 1000  # Se establece el número de pasos iniciales de recolección
for _ in range(initial_collect_steps):
    collect_step(train_env, random_policy, replay_buffer)  # Se recolecta experiencia utilizando la política aleatoria

# Se prepara el conjunto de datos
dataset = replay_buffer.as_dataset(  # Se convierte el buffer en un conjunto de datos
    num_parallel_calls=3,  # Se establece el número de llamadas paralelas
    sample_batch_size=32,  # Se define el tamaño del lote de muestra
    num_steps=2  # Se especifica el número de pasos a considerar en cada muestra
).prefetch(3)  # Se pre-carga el conjunto de datos
iterator = iter(dataset)  # Se crea un iterador para el conjunto de datos

print('Done')

In [ ]:
# Entrenamiento del agente
num_iterations = 200000  # Se ajusta este valor según los límites computacionales de Kaggle
collect_steps_per_iteration = 1  # Se define el número de pasos de recolección por iteración
log_interval = 1000  # Se establece el intervalo para los registros

for iteration in range(num_iterations):
    # Recolectar experiencia
    for _ in range(collect_steps_per_iteration):
        collect_step(train_env, agent.collect_policy, replay_buffer)  # Se llama a la función para recolectar experiencias

    # Muestra una experiencia del buffer y entrena al agente
    experience, _ = next(iterator)  # Se extrae un lote de experiencia del buffer
    train_loss = agent.train(experience).loss  # Se entrena al agente y se obtiene la pérdida



    if iteration % log_interval == 0:  # para cada intervalo
        print(f'Iteración: {iteration}, Pérdida: {train_loss}')  # Se imprime la iteración y la pérdida




In [ ]:
# # Evaluar el rendimiento del agente
# log_interval = 500  # Se define el intervalo para registrar información
# num_eval_episodes = 5  # Se establece el número de episodios de evaluación

# for episode in range(num_eval_episodes):  # Se itera sobre el número de episodios de evaluación
#     time_step = eval_env.reset()  # Se reinicia el entorno de evaluación
#     episode_reward = 0  # Se inicializa la recompensa total del episodio
#     i = 0  # Se inicializa un contador de iteraciones

#     while not time_step.is_last():  # Se ejecuta mientras no se alcance el final del episodio
#         action_step = agent.policy.action(time_step)  # Se selecciona una acción según la política del agente
#         time_step = eval_env.step(action_step.action)  # Se avanza en el entorno con la acción seleccionada
#         episode_reward += time_step.reward  # Se acumula la recompensa del paso actual
#         i += 1  # Se incrementa el contador de iteraciones

#         if i % log_interval == 0:  # Se verifica si se alcanza el intervalo de registro
#             print(f'Episode {episode + 1}: Ite.: {i} :episode_reward : {episode_reward.numpy()[0]}')  # Se imprime la recompensa acumulada


In [ ]:
# Configurar una pantalla virtual para renderizar entornos de OpenAI Gym
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()  # Se inicia la pantalla virtual

# Funciones de visualización proporcionadas
def update_scene(num, frames, patch):  # Se define la función para actualizar la escena
    patch.set_data(frames[num])  # Se actualizan los datos del frame actual
    return patch  # Se devuelve el parche actualizado

def plot_animation(frames, repeat=False, interval=40):  # Se define la función para crear la animación
    fig = plt.figure()  # Se crea una nueva figura
    patch = plt.imshow(frames[0])  # Se muestra el primer frame
    plt.axis('off')  # Se ocultan los ejes
    anim = animation.FuncAnimation(  # Se crea la animación
        fig, update_scene, fargs=(frames, patch),
        frames=len(frames), repeat=repeat, interval=interval)
    plt.close()  # Se cierra la figura para no mostrarla dos veces
    return anim  # Se devuelve la animación creada

# Recoger frames para la visualización
def run_and_visualize(agent, env):
    frames = []
    time_step = env.reset()
    policy_state = agent.policy.get_initial_state(env.batch_size)
    j = 0
    while not time_step.is_last():
    #for j in range(10000):
        action_step = agent.policy.action(time_step, policy_state)
        policy_state = action_step.state
        time_step = env.step(action_step.action)
        frame = np.squeeze(env.render())
        frames.append(frame)

        if j % 2000 == 0:


            print(f"Step: {len(frames)}, Action: {action_step.action}, Frame shape: {frame.shape}")
        j += 1



    print(f"Total frames collected: {len(frames)}")
    return frames

# Ejecutar el agente y recoger marcos
frames = run_and_visualize(agent, eval_env)  # Se ejecuta la función para recoger marcos

# Crear y mostrar la animación
anim = plot_animation(frames)  # Se crea la animación a partir de los marcos

# Mostrar la animación en Jupyter Notebook
from IPython.display import HTML  # Se importa la librería necesaria
HTML(anim.to_jshtml())  # Se convierte la animación a formato HTML y se muestra


In [ ]:
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

**Mario aleatorio****

In [ ]:
env = gym_super_mario_bros.make('SuperMarioBros-v2')
env = JoypadSpace(env, COMPLEX_MOVEMENT)
print(env.get_action_meanings())
state = env.reset()
env.step(env._action_space.sample())
img = env.render(mode="rgb_array")
plt.figure(figsize=(4, 6))
plt.imshow(img)
plt.axis("off")
plt.show()

# Create and wrap the environment
def create_environment1():
    # Load the Super Mario Bros environment
    env = gym_super_mario_bros.make('SuperMarioBros-v1')
    # Simplify the action space
    env = JoypadSpace(env, SIMPLE_MOVEMENT)
    # Apply preprocessing wrappers
    env = gym.wrappers.GrayScaleObservation(env, keep_dim=True)
    env = gym.wrappers.ResizeObservation(env, 84)
    env = gym.wrappers.FrameStack(env, 4)
    env = gym_wrapper.GymWrapper(env)
    return env

eval_py_env1 = create_environment1()

eval_env1 = tf_py_environment.TFPyEnvironment(eval_py_env1)
# Configurar una pantalla virtual para renderizar entornos de OpenAI Gym
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()  # Se inicia la pantalla virtual

# Funciones de visualización proporcionadas
def update_scene(num, frames, patch):  # Se define la función para actualizar la escena
    patch.set_data(frames[num])  # Se actualizan los datos del frame actual
    return patch  # Se devuelve el parche actualizado

def plot_animation1(frames, repeat=False, interval=40):  # Se define la función para crear la animación
    fig = plt.figure()  # Se crea una nueva figura
    patch = plt.imshow(frames[0])  # Se muestra el primer frame
    plt.axis('off')  # Se ocultan los ejes
    anim = animation.FuncAnimation(  # Se crea la animación
        fig, update_scene, fargs=(frames, patch),
        frames=len(frames), repeat=repeat, interval=interval)
    plt.close()  # Se cierra la figura para no mostrarla dos veces
    return anim  # Se devuelve la animación creada
def random_policy():
    return np.random.choice(len(SIMPLE_MOVEMENT))
# Recoger frames para la visualización
def run_and_visualize1(env):
    frames = []
    time_step = env.reset()
    #policy_state = agent.policy.get_initial_state(env.batch_size)

    for i in range(8000):
        action_step = random_policy()
        #policy_state = action_step.state
        time_step = env.step(action_step)

        # Obtener el frame y asegurarse de que es un array numpy
        frame = np.squeeze(env.render())

        frames.append(frame)
        i += 1

    print(f"Total frames collected: {len(frames)}")
    return frames


# Ejecutar el agente y recoger marcos
frames_random = run_and_visualize1(eval_env1)  # Se ejecuta la función para recoger marcos

# Crear y mostrar la animación
anim = plot_animation1(frames_random)  # Se crea la animación a partir de los marcos

# Mostrar la animación en Jupyter Notebook
from IPython.display import HTML  # Se importa la librería necesaria
HTML(anim.to_jshtml())  # Se convierte la animación a formato HTML y se muestra
